In [1]:
using Plots

n,p = 1000, 0.51
a,b = 1,1
f = 0.1

function gain(f,n=n, p=p, a=1,b=1)
    gains = cumsum(rand(n).<p)
    trials = 1:n
    fac = ((1 + f*a) / (1 - f*b)).*ones(n) 
    ret = fac.^gains
    return ret .* ( (1 - f*b).^trials )
end


R(f, p, a, b) = (1 + f*a)^p * (1 - f*b)^(1-p)
f_kelly(p,a,b) = p/b - (1-p)/a

f_kelly (generic function with 1 method)

In [2]:
n,N = 100, 10000
p,a,b = 0.7,1,1
f_list = 0.02:0.02:0.98

function samples(f,p,a,b)
    A = zeros(n,N)
    for i in 1:N
        A[:,i] = gain(f,n,p,a,b)
    end
    return A
end

D = Dict()
for f in f_list
    D[f] = samples(f,p,a,b)
end

In [3]:
plot(yscale=:log)
for f in f_list
    plot!(D[f][:,1:10], color=:red, alpha=0.2, seriestype=:steppost)
    plot!(sum(D[f],dims=2)/N, color=:black, seriestype=:steppost)
end
plot!(legend=:none)
ylims!((1e-30,1e20))

In [ ]:
function mean_floored(v)
    m = sum(v)./size(v)[1]
    ( isnan(m) || m<=1e-30 ) ? 1e-30 : m
end
M = [mean_floored(D[f][end,:]) for f in f_list];
using Statistics
σ = [std(D[f][end,:]) for f in f_list];

In [212]:
function g(f,p,a,b,n)
    gamma = (1 + f*a) / (1 - f*b)
    (1 - f*b)^n * gamma^(n*p) + 2
end

N_plot = maximum(N,1000)

vline([f_kelly(p,a,b)], color=:gray, lw=4, label="kelly's optimum")
for f in f_list
    scatter!(f.*ones(N_plot), D[f][end,1:N_plot], color=:lightsalmon, alpha=0.2, ms=2, label=:none, markerstrokewidth=0)
end
plot!(yscale=:log)
plot!(ylims=(1e-20,1e20))
e = p*a - (1-p)*b
plot!(f_list, (1 .+ e.*f_list).^n , lw=2,markershape=:circle, markersize=4, markerstrokewidth=0, label="dummy gain")
plot!(f_list, R.(f_list, p,a,b).^n, lw=2, markershape=:circle, markersize=4, markerstrokewidth=0, label="kelly gain")

scatter!(f_list, M, color=:lightsalmon, label="mean gain over 100 runs")
#plot!(f_list, M .+ σ, fillrange = M.-σ, color=:gray, alpha=0.2)




MethodError: MethodError: objects of type Int64 are not callable
Maybe you forgot to use an operator such as *, ^, %, / etc. ?